In [1]:
import sys
import xarray as xr
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import glob
from netCDF4 import Dataset
import cartopy.crs as ccrs
import os

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
%autosave 60

/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


Autosaving every 60 seconds


In [2]:
sys.path.insert(0,"/scratch/cnt0024/hmg2840/albert7a/DEV/git/powerspec/powerspec")
import powerspec as pp

In [3]:
sys.path.insert(0,"/scratch/cnt0024/hmg2840/albert7a/DEV/git/diags-CMEMS-on-occigen/common-lib/")
import GriddedData
import WavenumberSpectrum as ws


In [4]:
# Some functions that allow us to compute spectra in boxes

##########################################
def get_values_in_box(imin,imax,jmin,jmax,data):
    values = data[:,jmin:jmax+1,imin:imax+1]
    values = ma.masked_invalid(values)
    return values


##########################################
def compute_spec_for_box(imin,imax,jmin,jmax,data,Mth,navlon,navlat,name,WaveSpecResult,param):
    var = get_values_in_box(imin,imax,jmin,jmax,data)
    pspec,kspec = compute_spec(var,navlon,navlat)
    np.savez(WaveSpecResult+'WaveSpec_'+name+'_'+param+'_'+Mth, kspec=kspec ,pspec=pspec)

##########################################
def compute_spec(data,navlon,navlat):
    days = len(data)
    mth_pspec = []
    for it in np.arange(0,days):
        arr = data[it]
        datab = arr.squeeze()
        pspec,kstep = pp.wavenumber_spectra(datab,navlon,navlat)
        mth_pspec.append(pspec)
    mthly_pspec = np.array(mth_pspec)
    mean_mthly_pspec = mthly_pspec.mean(axis=0)
    return kstep,mean_mthly_pspec

In [5]:
def compute_uspec_eu(box):
    
    if box == 'box1':
        imin=125
        imax=427
        jmin=165
        jmax=544
    if box == 'box2':
        imin=116
        imax=421
        jmin=540
        jmax=948
    if box == 'box3':
        imin=99
        imax=409
        jmin=939
        jmax=1378
    if box == 'box4':
        imin=35
        imax=348
        jmin=1816
        jmax=2321

    param='Uspec'
    varname = 'vozocrtx'
    WaveSpecResult = '/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60EU-'+str(param)+'/'
    !mkdir -p $WaveSpecResult
    YrMth  = ['2013m01','2013m02','2013m03','2013m04','2013m05','2013m06','2013m07','2013m08','2013m09','2012m10','2012m11','2012m12']

    for ii in np.arange(12):
        Mth = YrMth[ii]
        if not os.path.exists(WaveSpecResult+'WaveSpec_'+str(box)+'_'+param+'_'+Mth+'.npz'):
            print(WaveSpecResult+'WaveSpec_'+str(box)+'_'+param+'_'+Mth+'.npz')
            filename='/scratch/cnt0024/hmg2840/albert7a/NATL60/NATL60-CJM165-S/1d/EU/NATL60EU-CJM165_y'+str(YrMth[ii])+'d??.1d_gridUsurf.nc'
            ds = xr.open_mfdataset(filename,chunks={'x':500,'y':500,'time_counter':1})
            data = ds[varname]
            if len(ds.nav_lon.dims) == 3:
                navlon=ds.nav_lon[0,jmin:jmax+1,imin:imax+1]
                navlat=ds.nav_lat[0,jmin:jmax+1,imin:imax+1]
            if len(ds.nav_lon.dims) == 2:
                navlon=ds.nav_lon[jmin:jmax+1,imin:imax+1]
                navlat=ds.nav_lat[jmin:jmax+1,imin:imax+1]
                
            compute_spec_for_box(imin,imax,jmin,jmax,data,Mth,navlon,navlat,box,WaveSpecResult,param)


In [6]:
def compute_vspec_eu(box):
    
    if box == 'box1':
        imin=125
        imax=427
        jmin=165
        jmax=544
    if box == 'box2':
        imin=116
        imax=421
        jmin=540
        jmax=948
    if box == 'box3':
        imin=99
        imax=409
        jmin=939
        jmax=1378
    if box == 'box4':
        imin=35
        imax=348
        jmin=1816
        jmax=2321

    param='Vspec'
    varname = 'vomecrty'
    WaveSpecResult = '/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60EU-'+str(param)+'/'
    !mkdir -p $WaveSpecResult
    YrMth  = ['2013m01','2013m02','2013m03','2013m04','2013m05','2013m06','2013m07','2013m08','2013m09','2012m10','2012m11','2012m12']

    for ii in np.arange(12):
        Mth = YrMth[ii]
        if not os.path.exists(WaveSpecResult+'WaveSpec_'+str(box)+'_'+param+'_'+Mth+'.npz'):
            print(WaveSpecResult+'WaveSpec_'+str(box)+'_'+param+'_'+Mth+'.npz')
            filename='/scratch/cnt0024/hmg2840/albert7a/NATL60/NATL60-CJM165-S/1d/EU/NATL60EU-CJM165_y'+str(YrMth[ii])+'d??.1d_gridVsurf.nc'
            ds = xr.open_mfdataset(filename,chunks={'x':500,'y':500,'time_counter':1})
            data = ds[varname]
            if len(ds.nav_lon.dims) == 3:
                navlon=ds.nav_lon[0,jmin:jmax+1,imin:imax+1]
                navlat=ds.nav_lat[0,jmin:jmax+1,imin:imax+1]
            if len(ds.nav_lon.dims) == 2:
                navlon=ds.nav_lon[jmin:jmax+1,imin:imax+1]
                navlat=ds.nav_lat[jmin:jmax+1,imin:imax+1]
            compute_spec_for_box(imin,imax,jmin,jmax,data,Mth,navlon,navlat,box,WaveSpecResult,param)


In [7]:
compute_uspec_eu('box1')
compute_vspec_eu('box1')


In [8]:
def compute_kespec(box):
    YrMth  = ['2013m01','2013m02','2013m03','2013m04','2013m05','2013m06','2013m07','2013m08','2013m09','2012m10','2012m11','2012m12']
    # Folders containing U and V spectral
    u_database = '/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60EU-Uspec/'
    v_database = '/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60EU-Vspec/'    # Folders to contain computed KE spectral
    KEspecFolder = '/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60EU-KEspec/'    
    !mkdir -p $KEspecFolder
    u_filenames = sorted(glob.glob(u_database + 'WaveSpec_'+str(box)+'_Uspec_*.npz'))
    v_filenames = sorted(glob.glob(v_database + 'WaveSpec_'+str(box)+'_Vspec_*.npz'))
    for i in range(len(u_filenames)):
        if not os.path.exists(KEspecFolder+'WaveSpec_'+str(box)+'_KEspec_'+YrMth[i]+'.npz'):
            ufile = u_database + 'WaveSpec_'+str(box)+'_Uspec_'+YrMth[i]+'.npz'
            vfile = v_database + 'WaveSpec_'+str(box)+'_Vspec_'+YrMth[i]+'.npz'
            print(ufile)
            uspec = np.load(ufile)['pspec']
            print(vfile)
            vspec = np.load(vfile)['pspec']
            kspec = np.load(ufile)['kspec']
            KEspec = 0.5*(uspec + vspec)
            print(KEspecFolder+'WaveSpec_'+str(box)+'_KEspec_'+YrMth[i]+'.npz')
            np.savez(KEspecFolder+'WaveSpec_'+str(box)+'_KEspec_'+YrMth[i]+'.npz',kspec = kspec,KEspec = KEspec)

In [9]:
compute_kespec('box1')

In [10]:
compute_uspec_eu('box2')
compute_vspec_eu('box2')
compute_kespec('box2')

In [11]:
compute_uspec_eu('box3')
compute_vspec_eu('box3')
compute_kespec('box3')

/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60EU-Vspec/WaveSpec_box3_Vspec_2012m10.npz
/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60EU-Vspec/WaveSpec_box3_Vspec_2012m11.npz
/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60EU-Vspec/WaveSpec_box3_Vspec_2012m12.npz
/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60EU-Uspec/WaveSpec_box3_Uspec_2013m01.npz
/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60EU-Vspec/WaveSpec_box3_Vspec_2013m01.npz
/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60EU-KEspec/WaveSpec_box3_KEspec_2013m01.npz
/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60EU-Uspec/WaveSpec_box3_Uspec_2013m02.npz
/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60EU-Vspec/WaveSpec_box3_Vspec_2013m02.npz
/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60EU-KEspec/WaveSpec_box3_KEspec_2013m02.npz
/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60EU-Uspec/WaveSpec_box3_Uspec_2013m03.npz
/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60EU-Vspec/WaveS

In [ ]:
compute_uspec_eu('box4')
compute_vspec_eu('box4')
compute_kespec('box4')

/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60EU-Uspec/WaveSpec_box4_Uspec_2013m01.npz
/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60EU-Uspec/WaveSpec_box4_Uspec_2013m02.npz
/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60EU-Uspec/WaveSpec_box4_Uspec_2013m03.npz
/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60EU-Uspec/WaveSpec_box4_Uspec_2013m04.npz
/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60EU-Uspec/WaveSpec_box4_Uspec_2013m05.npz
